# JobFitScore com Ollama (local) e Fallback (Transformers/Colab)

Este notebook suporta dois modos:

- Modo local (Ollama): USE_OLLAMA = True. Requer o Ollama rodando em http://localhost:11434
- Modo Colab (Fallback Transformers): USE_OLLAMA = False. Usa Hugging Face Transformers com GPU do Colab.

Observação: Rodar Ollama no Colab não é recomendado. Prefira o fallback com Transformers no Colab.


## 1. Instalar bibliotecas necessárias (modo Colab)

Execute esta célula apenas se estiver no Google Colab. No ambiente local com Ollama já instalado, não é necessário.


In [ ]:
# Detecta se está em Colab e instala dependências se necessário
try:
    import google.colab  # type: ignore
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

if IS_COLAB:
    !pip install -q transformers accelerate sentencepiece requests
else:
    print("Ambiente local detectado. Pulando instalação de pacotes Colab.")

## 1.5 Instalar biblioteca para leitura de PDF

In [ ]:
# Instala PyPDF2 para ler PDFs (funciona tanto local quanto no Colab)
try:
    import PyPDF2
    print("PyPDF2 já está instalado.")
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "PyPDF2"])
    import PyPDF2
    print("PyPDF2 instalado com sucesso.")

## 2. Importar bibliotecas e configurar ambiente


In [ ]:
import os, sys, io, json, re
import PyPDF2

# requests pode não estar no ambiente local se não instalado
try:
    import requests
except ImportError:
    requests = None

# Imports condicionais para Transformers (Colab)
if 'IS_COLAB' in globals() and IS_COLAB:
    from transformers import pipeline
    from google.colab import files

# Força stdout UTF-8 (especialmente em Windows)
if hasattr(sys.stdout, 'buffer'):
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

# Flags e configurações
USE_OLLAMA = not IS_COLAB  # Se estiver em Colab, usa fallback Transformers
OLLAMA_URL = 'http://localhost:11434/api/generate'
MODEL_NAME = 'llama3.2:3b'  # Ajuste para modelos maiores locais: 'gemma3:27b'
TRANSFORMERS_MODEL = 'google/gemma-2-2b-it'  # Modelo HF para fallback

print(f"IS_COLAB={IS_COLAB} | USE_OLLAMA={USE_OLLAMA}")

## 3. Funções auxiliares (definir antes de usar)

## 3.1. Funções de geração com IA (Ollama e Transformers)

In [ ]:
def gerar_com_ollama(prompt_text: str, model: str = MODEL_NAME):
    """Chama a API local do Ollama e retorna texto JSON."""
    if requests is None:
        raise RuntimeError("'requests' não disponível.")
    payload = {
        "model": model,
        "prompt": prompt_text,
        "stream": False,
        "format": "json",
        "options": {
            "num_ctx": 4096,
            "temperature": 0.2,
            "num_predict": 800,
        },
    }
    try:
        resp = requests.post(OLLAMA_URL, json=payload, timeout=180)
        resp.raise_for_status()
        data = resp.json()
        return data.get("response", "")
    except requests.exceptions.ConnectionError:
        raise ConnectionError("Não conectou ao Ollama. Certifique-se de que o servidor está ativo.")
    except requests.exceptions.Timeout:
        raise TimeoutError("Timeout: o modelo demorou para responder.")
    except requests.exceptions.HTTPError as e:
        raise e
    except Exception as e:
        raise RuntimeError(f"Erro inesperado na chamada Ollama: {e}")


def gerar_com_transformers(prompt_text: str, model_id: str = TRANSFORMERS_MODEL):
    """Usa pipeline HF para gerar texto e extrair JSON."""
    if not IS_COLAB:
        raise RuntimeError("Fallback Transformers destinado ao modo Colab.")
    pipe = pipeline(
        "text-generation",
        model=model_id,
        device_map="auto",
        torch_dtype="auto",
        max_new_tokens=700,
        temperature=0.2,
    )
    output = pipe(prompt_text)
    texto = output[0]["generated_text"]
    # Extrai JSON bruto (heurística simples)
    match = re.search(r"\{.*\}\s*$", texto, re.DOTALL)
    if match:
        json_text = match.group(0)
        try:
            return json.loads(json_text)
        except json.JSONDecodeError:
            raise ValueError("Não foi possível decodificar JSON do resultado Transformers.")
    else:
        raise ValueError("JSON não encontrado na saída do modelo Transformers.")

print("Funções gerar_com_ollama e gerar_com_transformers definidas.")

## 3.2. Função para extrair texto de PDF

In [ ]:
def extrair_texto_pdf(caminho_pdf: str) -> str:
    """Extrai todo o texto de um arquivo PDF."""
    texto_completo = ""
    try:
        with open(caminho_pdf, 'rb') as arquivo:
            leitor = PyPDF2.PdfReader(arquivo)
            for pagina in leitor.pages:
                texto_completo += pagina.extract_text() + "\n"
        return texto_completo.strip()
    except Exception as e:
        raise RuntimeError(f"Erro ao ler PDF: {e}")

print("Função extrair_texto_pdf definida.")

## 3.3. Função para estruturar dados do currículo usando IA

In [ ]:
def processar_curriculo(texto_curriculo: str) -> dict:
    """Usa a IA para extrair informações estruturadas do currículo."""
    prompt_extracao = f"""
Analise o currículo abaixo e extraia as seguintes informações em formato JSON:

{{
  "nome": "nome completo do candidato",
  "habilidades": ["lista", "de", "habilidades", "técnicas"],
  "experiencia": "resumo breve da experiência profissional em uma frase",
  "cursos": ["lista", "de", "cursos", "ou", "certificações"]
}}

IMPORTANTE: Retorne APENAS o JSON, sem markdown ou explicações.

Currículo:
{texto_curriculo}
"""
    
    try:
        if USE_OLLAMA:
            resposta = gerar_com_ollama(prompt_extracao, model=MODEL_NAME)
            candidato = json.loads(resposta)
        else:
            candidato = gerar_com_transformers(prompt_extracao, model_id=TRANSFORMERS_MODEL)
        
        # Valida campos obrigatórios
        campos = ['nome', 'habilidades', 'experiencia', 'cursos']
        for campo in campos:
            if campo not in candidato:
                raise ValueError(f"Campo '{campo}' não encontrado no JSON extraído.")
        
        return candidato
    except Exception as e:
        raise RuntimeError(f"Erro ao processar currículo: {e}")

print("Função processar_curriculo definida.")

## 3.4. Upload de currículo (Colab) ou seleção de arquivo (local)

In [ ]:
# MODO DE USO:
# - No Colab: execute esta célula e faça upload do PDF quando solicitado
# - Local: defina CAMINHO_PDF com o caminho do arquivo PDF ou deixe None para usar dados de exemplo

# Opção 1: Usar arquivo PDF do candidato
CAMINHO_PDF = None  # Ex: "curriculo_joao.pdf" (local) ou None para upload no Colab

# Opção 2: Usar dados de exemplo (descomente a linha abaixo para desabilitar upload)
# USAR_DADOS_EXEMPLO = True

USAR_DADOS_EXEMPLO = False

if not USAR_DADOS_EXEMPLO:
    if IS_COLAB and CAMINHO_PDF is None:
        # Upload no Colab
        print("Faça upload do currículo em PDF:")
        uploaded = files.upload()
        CAMINHO_PDF = list(uploaded.keys())[0]
        print(f"Arquivo recebido: {CAMINHO_PDF}")
    elif CAMINHO_PDF is None:
        # Local: solicita o caminho
        CAMINHO_PDF = input("Digite o caminho completo do PDF do currículo: ").strip('"').strip("'")
    
    # Processa o PDF
    print(f"\nProcessando currículo: {CAMINHO_PDF}...")
    texto_cv = extrair_texto_pdf(CAMINHO_PDF)
    print(f"Texto extraído ({len(texto_cv)} caracteres)")
    print(f"\nPrimeiros 300 caracteres:\n{texto_cv[:300]}...\n")
    
    print("Extraindo informações estruturadas com IA...")
    candidato_novo = processar_curriculo(texto_cv)
    print(f"Candidato processado: {candidato_novo['nome']}")
    
    # Adiciona o novo candidato à lista
    candidatos_lista = [candidato_novo]
else:
    # Usa dados de exemplo
    candidatos_lista = [
        {
            "nome": "Ana Souza",
            "habilidades": ["React Native", "JavaScript", "Figma", "UX Design", "Git"],
            "experiencia": "2 anos como desenvolvedora mobile em React Native",
            "cursos": ["React Native Avançado", "Design de Interfaces"],
        },
        {
            "nome": "Lucas Pereira",
            "habilidades": ["JavaScript", "TypeScript", "Node.js", "ReactJS"],
            "experiencia": "3 anos como desenvolvedor full-stack, iniciando com React Native",
            "cursos": ["ReactJS Completo", "APIs REST com Node.js"],
        },
        {
            "nome": "Mariana Lima",
            "habilidades": [
                "HTML",
                "CSS",
                "React Native",
                "APIs REST",
                "Git",
                "TypeScript",
            ],
            "experiencia": "1 ano como estagiária em desenvolvimento mobile",
            "cursos": ["Introdução ao React Native", "Versionamento com Git"],
        },
    ]
    print("Usando dados de exemplo (3 candidatos).")

print(f"\nTotal de candidatos: {len(candidatos_lista)}")

In [ ]:
# Define a vaga
vaga = {
    "titulo": "Desenvolvedor Front-End React Native",
    "empresa": "TechFlow Solutions",
    "requisitos": [
        "React Native",
        "JavaScript",
        "TypeScript",
        "APIs REST",
        "Git",
        "UI/UX básico",
    ],
    "descricao": "Responsável por desenvolver e manter aplicativos móveis usando React Native, garantindo performance e boa experiência do usuário.",
}

# Monta o dicionário de dados completo
dados = {
    "vaga": vaga,
    "candidatos": candidatos_lista  # Preenchido na célula anterior
}

print(f"Vaga: {vaga['titulo']}")
print(f"Candidatos: {len(dados['candidatos'])}")

## 4. Montagem do prompt


In [ ]:
prompt = f"""
Você é um avaliador técnico de compatibilidade entre candidatos e vagas de emprego.

Analise os dados abaixo em formato JSON. Compare as habilidades, experiências e cursos dos candidatos com os requisitos da vaga.

Para cada candidato, calcule um score de compatibilidade de 0 a 100 e retorne em formato JSON no seguinte modelo:

{{
  "avaliacoes": [
    {{
      "nome": "Nome do candidato",
      "score": número,
      "feedback": "breve explicação sobre a pontuação"
    }}
  ]
}}

Use os seguintes critérios:
- + pontos para cada habilidade que coincidir com os requisitos da vaga.
- Considere experiência e cursos relacionados como fator positivo.
- Diminua pontos se o candidato não tiver tecnologias essenciais da vaga.
- O score deve refletir a chance real de sucesso na vaga (0 a 100).

IMPORTANTE: Retorne APENAS o JSON, sem markdown ou explicações adicionais.

Dados:
{json.dumps(dados, ensure_ascii=False, indent=2)}
"""
print("Prompt pronto.")

## 5. Executar avaliação e salvar resultado

In [ ]:
def exibir(resultado_dict: dict):
    print("=" * 60)
    print(f"VAGA: {dados['vaga']['titulo']}")
    print(f"EMPRESA: {dados['vaga']['empresa']}")
    print("=" * 60)
    for av in resultado_dict.get('avaliacoes', []):
        print(f"- {av.get('nome')}")
        print(f"  Score: {av.get('score')}/100")
        print(f"  Feedback: {av.get('feedback')}")
        print("-" * 60)

try:
    if USE_OLLAMA:
        print(f"Usando Ollama local | Modelo: {MODEL_NAME}")
        saida = gerar_com_ollama(prompt, model=MODEL_NAME)
        # Ollama retorna texto JSON; converter para dict
        try:
            resultado = json.loads(saida)
        except json.JSONDecodeError:
            raise ValueError("Resposta do Ollama não é um JSON válido.")
        destino = 'resultado_avaliacao_ollama.json'
    else:
        print(f"Usando Transformers (Colab) | Modelo: {TRANSFORMERS_MODEL}")
        resultado = gerar_com_transformers(prompt, model_id=TRANSFORMERS_MODEL)
        destino = 'resultado_avaliacao_transformers.json'

    exibir(resultado)
    with open(destino, 'w', encoding='utf-8') as f:
        json.dump(resultado, f, ensure_ascii=False, indent=2)
    print(f"\nResultado salvo em: {destino}")

except Exception as e:
    print(f"Erro durante a avaliação: {e}")
    if USE_OLLAMA:
        print("Dica: verifique se o Ollama está rodando e se o modelo foi baixado (ollama list / ollama pull).")

## 6. Dicas para ajuste de modelos

- Modelos maiores locais: `llama3.1:8b`, `mistral:7b-instruct`, `gemma3:27b` (requer mais RAM/VRAM).
- Em Colab, prefira: `google/gemma-2-2b-it`, `microsoft/Phi-3-mini-4k-instruct`.
- Ajuste `num_ctx` para janelas maiores (custo de memória). Ajuste `num_predict` para limitar saída.
- Reduza `temperature` para respostas mais consistentes.

## 7. Como usar com seu próprio currículo

### Modo Local (Windows/macOS/Linux):
1. Salve seu currículo como PDF (ex: `meu_curriculo.pdf`)
2. Na célula "3.4. Upload de currículo", defina:
   ```python
   CAMINHO_PDF = "C:/caminho/completo/meu_curriculo.pdf"
   USAR_DADOS_EXEMPLO = False
   ```
3. Execute todas as células na ordem

### Modo Colab:
1. Execute as células 1, 1.5, 2 e 3.1 primeiro
2. Na célula "3.4. Upload de currículo", defina:
   ```python
   USAR_DADOS_EXEMPLO = False
   ```
3. Execute a célula - aparecerá um botão de upload
4. Selecione seu PDF
5. Continue executando as células seguintes

### Para testar com dados de exemplo:
```python
USAR_DADOS_EXEMPLO = True
```